In [2]:
import imaplib
import email
from email.header import decode_header
from email.utils import parsedate_tz, mktime_tz, parseaddr
from email import policy
import openpyxl

# Configuración
imap_server = "imap.gmail.com"
email_user = "ljavierminguillan@gmail.com"
email_pass = "kogu lbfj taof fdwj"

# Conexión al servidor IMAP de Gmail
mail = imaplib.IMAP4_SSL(imap_server)
mail.login(email_user, email_pass)

# Seleccionar la bandeja de entrada
mail.select("inbox")

# Busca los últimos 100 correos
status, messages = mail.search(None, 'ALL')
message_ids = messages[0].split()
last_100_ids = message_ids[-100:]  # Últimos 100 correos

# Crea un archivo Excel
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Emails"
ws.append(["Msg_ID", "De (Nombre)", "Correo (De)", "Asunto", "Hilo (In-Reply-To)", "Hora de Envío", "CC", "Cuerpo del Mensaje", "Leído", "Etiquetas"])  # Encabezados

# Procesar los correos
emails_data = []  # Lista para almacenar los datos temporalmente
for msg_id in last_100_ids:
    # Obtén el correo y los flags en una sola solicitud
    status, msg_data = mail.fetch(msg_id, "(RFC822 FLAGS)")
    if status != "OK":
        continue

    for response_part in msg_data:
        if isinstance(response_part, tuple):
            # Convierte el correo en un objeto Message
            msg = email.message_from_bytes(response_part[1], policy=policy.default)

            # Decodifica el asunto
            subject, encoding = decode_header(msg["Subject"])[0]
            subject = subject.decode(encoding if encoding else "utf-8") if isinstance(subject, bytes) else subject

            # Obtén el remitente
            from_ = parseaddr(msg.get("From"))
            from_name, from_email = from_[0], from_[1]

            # Obtén la fecha de envío
            date_tuple = parsedate_tz(msg["Date"])
            date_sent = email.utils.formatdate(mktime_tz(date_tuple)) if date_tuple else "Desconocida"

            # Obtén los destinatarios en CC
            cc = msg.get("Cc", "No hay CC")

            # Obtén el hilo (In-Reply-To)
            in_reply_to = msg.get("In-Reply-To", "No hay hilo")

            # Obtener el cuerpo del mensaje
            body = ""
            if msg.is_multipart():
                for part in msg.iter_parts():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode(part.get_content_charset(), errors="ignore")
                        break  # Solo necesitamos un texto plano
            else:
                if msg.get_content_type() == "text/plain":
                    body = msg.get_payload(decode=True).decode(msg.get_content_charset(), errors="ignore")

            # Verificar si el correo ha sido leído
            flags = msg_data[1].decode()

            # Extraer etiquetas del correo desde los flags
            labels = [flag for flag in flags.split()]

            # Almacenar los datos en memoria
            emails_data.append([
                msg['Message-ID'], from_name, from_email, subject, in_reply_to, 
                date_sent, cc, body[:500], ", ".join(labels)
            ])

# Agregar todos los datos al archivo Excel
for email_data in emails_data:
    ws.append(email_data)

# Guardar el archivo Excel
wb.save("últimos_100_correos_rapido.xlsx")

# Cerrar la conexión
mail.close()
mail.logout()


('BYE', [b'LOGOUT Requested'])